In [1]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import pysal.model.spreg as spreg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal.lib
import geopandas as gpd
import pandas as pd
from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
import pandas as pd
from scipy.stats import pearsonr
from esda.moran import Moran
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import spreg
from spreg import SURlagIV
from spreg import GM_Lag
import libpysal
from spreg import AKtest
from spreg import ML_Lag
from spreg import GM_Combo

**Loading Path**

In [2]:
# Dependent variable
dep_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

# Independent variable
ind_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\1.2 Saving the Independent Variables"

# Steamengines and horsepower
power_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

In [3]:
# lets read the metadata
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\control.dta") as reader:
    meta_control = reader.variable_labels()
    
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\_FINAL_c_steamenginges_1904.dta") as reader:
    meta_steam = reader.variable_labels()

In [4]:
meta_control

{'id_inequality': 'County ID',
 'rb_id': 'District ID',
 'rb_name': 'District name',
 'province_id': 'Province ID',
 's_industry': 'Share industry employment',
 's_prot': 'Share Protestants',
 's_german': 'Share German-speaking',
 'longitud': 'Longitude',
 'latitud': 'Latitude',
 'slope_km': 'Terrain slope',
 'soil_suitability': 'Soil suitability',
 'mean_temp': 'Average temperature',
 'mean_preci': 'Average precipitation',
 'dist_100km': 'Distance to the nearest river',
 'log_distance_carbon_c': 'Least distance to carbon field (in log)'}

In [5]:
# Load Skill levels per Landkreis
skills_df = pd.read_csv(f"{dep_var_path}\Main_Dep_Diff_Skills.csv",
                        sep=",")

# Load control variables
control_df = pd.read_csv(f"{ind_var_path}\Independent Variables.csv",
                         sep=",")

# Load SteamEngine and Horsepower data
power_df = pd.read_stata(f"{power_path}\_FINAL_c_steamenginges_1904.dta")

In [6]:
# Merge the dataframes using an inner join on the "id_inequality" column
prep_df = pd.merge(control_df, skills_df, on="id_inequality", how="inner")
merged_df = pd.merge(prep_df, power_df, on="id_inequality", how="inner")

In [7]:
# Transform steamengines to steamengines per worker
merged_df["steamengines_number"] = (merged_df["steamengines_number"] / merged_df["d_Hauptberuf"])*1000
merged_df["horsepower"] = (merged_df["horsepower"] / merged_df["d_Hauptberuf"])*1000

merged_df["steamengines_number"] = np.log(merged_df["steamengines_number"])
merged_df["horsepower"] = np.log(merged_df["horsepower"])

In [8]:
merged_df["per_unskilled"] = merged_df["unskilled"] / merged_df["d_Hauptberuf"]
merged_df["per_low"] = merged_df["low"] / merged_df["d_Hauptberuf"]
merged_df["per_medium"] = merged_df["medium"] / merged_df["d_Hauptberuf"]
merged_df["per_high"] = merged_df["high"] / merged_df["d_Hauptberuf"]

merged_df["per_unskilled"] = np.log(merged_df["per_unskilled"])
merged_df["per_low"] = np.log(merged_df["per_low"])
merged_df["per_medium"] = np.log(merged_df["per_medium"])
merged_df["per_high"] = np.log(merged_df["per_high"])

In [9]:
# Drop Erblanden
#district_Erblanden = [31]
#merged_df = merged_df[~merged_df["rb_id"].isin(district_Erblanden)]

# Make Dummies for FE

In [10]:
dummies = pd.get_dummies(merged_df["province_id"], drop_first=True) * 1
merged_df = pd.concat([merged_df, dummies], axis=1)

# Convert column names to strings
merged_df.columns = [str(col) for col in merged_df.columns]

In [11]:
# Dictionary
number_words = {
    "1": "one", "2": "two", "3": "three", "4": "four", "5": "five",
    "6": "six", "7": "seven", "8": "eight", "9": "nine", "10": "ten",
    "11": "eleven", "12": "twelve", "13": "thirteen", "14": "fourteen"
}

# Transform the column names
merged_df.columns = [number_words[col.split('.')[0]] if col.split('.')[0] in number_words else col for col in merged_df.columns]

merged_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,five,six,seven,eight,nine,ten,eleven,twelve,thirteen,fourteen
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,0,0,0,0,0,0,0,0,0,0
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,0,0,0,0,0,0,0,0,0,0
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,0,0,0,0,0,0,0,0,0,0
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,0,0,0,0,0,0,0,0,0,0
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,0,0,0,0,0,0,0,1,0,0
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,0,0,0,0,0,0,0,0,0,1
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,0,0,0,0,0,0,0,0,0,1
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,0,0,0,0,0,0,0,0,0,1


# 2SLS
Lets run a good'ol reliable IV first

## 2SLS function
I build a function that runs multiple 2SLS in a row, each time adding another variable from a list of control variables

In [12]:
def twoSLS(df, IV, first_stage_dependent, controls, second_stage_independent):
    """
    Performs a series of Two-Stage Least Squares (2SLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    second_stage_independent (str): The name of the main independent variable for the second stage regression.

    Returns:
    tuple of DataFrames: The first DataFrame contains the results (coefficients, significance, 
                         Adj. R-squared, F-statistic) of each first stage regression. 
                         The second DataFrame contains the results 
                         of each second stage regression. Significance is denoted by stars: 
                         *** for p < 0.01, ** for p < 0.05, and * for p < 0.1.
                         The third DataFrame contains the residuals.
    """

    first_stage_results = []
    second_stage_results = []
    # For analysis of spatial autocorrelation
    residuals = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = " + ".join(subset_controls)
        if controls_formula_part:
            controls_formula_part = " + " + controls_formula_part

        # To avoid SettingWithCopyWarning
        df_copy = df.copy()

        # First Stage Regression
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()

        # Add the predicted values as a new column in the copy
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression
        second_stage_formula = f"{second_stage_independent} ~ predicted{controls_formula_part}"
        second_stage_model = smf.ols(second_stage_formula, data=df_copy)
        second_stage_fit = second_stage_model.fit()

        def format_coef(coef, std_err, p_value):
            stars = ""
            if p_value < 0.01:
                stars = "***"
            elif p_value < 0.05:
                stars = "**"
            elif p_value < 0.1:
                stars = "*"
    
            # Check if coef or std_err is smaller than 0.000 and format accordingly
            coef_str = f"{coef:.10e}" if abs(coef) < 0.000 else f"{coef:.3f}"
            std_err_str = f"{std_err:.10e}" if abs(std_err) < 0.000 else f"{std_err:.3f}"

            return f"{coef_str}{stars} ({std_err_str})"


        # Extract coefficients and significance of first stage
        first_stage_summary = first_stage_fit.summary2().tables[1]
        first_stage_coefs = first_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Extract coefficients and significance of second stage
        second_stage_summary = second_stage_fit.summary2().tables[1]
        second_stage_coefs = second_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Adding R-squared and F-statistic
        first_stage_coefs.loc["Adj. R-squared"] = f"{first_stage_fit.rsquared_adj:.3f}"
        first_stage_coefs.loc["F-statistic"] = f"{first_stage_fit.fvalue:.3f}"
        second_stage_coefs.loc["Adj. R-squared"] = f"{second_stage_fit.rsquared_adj:.3f}"
        
        # Adding Standard Errors
        second_stage_se = second_stage_fit.bse["predicted"]
        second_stage_coefs.loc["std err"] = second_stage_se
        
        # Residuals
        res = second_stage_fit.resid
        
        # Append 
        first_stage_results.append(first_stage_coefs)
        second_stage_results.append(second_stage_coefs)
        residuals.append(res)

    # Convert the results to DataFrame
    first_stage_df = pd.DataFrame(first_stage_results).T
    second_stage_df = pd.DataFrame(second_stage_results).T
    
    # Df of residuals
    residuals_df = pd.DataFrame(residuals).T

    return first_stage_df, second_stage_df, residuals_df

## Use the Function

In [13]:
# DataFrame that contains all the variables for the regression analysis
dep_var = merged_df[["log_distance_carbon_c", 
                     "s_industry",
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km",
                     "uni_dummy", 
                     "min_distance_uni", 
                     "prov_cap_dummy", 
                     "BER_dummy", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high",
                     "d_Hauptberuf",
                     "horsepower",
                     "two", 
                     "three", 
                     "four", 
                     "five", 
                     "six", 
                     "seven", 
                     "eight", 
                     "nine", 
                     "ten", 
                     "eleven", 
                     "twelve", 
                     "thirteen", 
                     "fourteen"]]

In [14]:
# Make two lists of exogenous variables
# Exclude latitude and longitude once
control_list = ["longitud",
                "latitud",
                "soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "two", 
                "three", 
                "four", 
                "five", 
                "six", 
                "seven", 
                "eight", 
                "nine", 
                "ten", 
                "eleven", 
                "twelve", 
                "thirteen", 
                "fourteen"]

control_list_2 = ["soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "two", 
                "three", 
                "four", 
                "five", 
                "six", 
                "seven", 
                "eight", 
                "nine", 
                "ten", 
                "eleven", 
                "twelve", 
                "thirteen", 
                "fourteen"]

In [15]:
first_unskilled, second_unskilled, res_unskilled = twoSLS(dep_var,
                                                          "log_distance_carbon_c",
                                                          "steamengines_number",
                                                          control_list,
                                                          "per_unskilled")

first_low, second_low, res_low = twoSLS(dep_var,
                                        "log_distance_carbon_c",
                                        "steamengines_number",
                                        control_list,
                                        "per_low")

first_medium, second_medium, res_medium = twoSLS(dep_var,
                                                 "log_distance_carbon_c",
                                                 "steamengines_number",
                                                 control_list,
                                                 "per_medium")

first_high, second_high, res_high = twoSLS(dep_var,
                                           "log_distance_carbon_c",
                                           "steamengines_number",
                                           control_list,
                                           "per_high")

In [16]:
first_unskilled[[1,6,8,24]]

,1,6,8,24
Intercept,2.646*** (0.075),-9.296*** (1.141),-8.241*** (1.266),-6.132** (2.391)
log_distance_carbon_c,-0.095*** (0.016),-0.189*** (0.020),-0.184*** (0.021),-0.169*** (0.023)
Adj. R-squared,0.053,0.255,0.259,0.351
F-statistic,17.174,29.136,23.333,13.914
longitud,0.008 (0.005),0.041*** (0.009),0.039*** (0.010),0.098*** (0.024)
latitud,NaN,0.175*** (0.022),0.155*** (0.026),0.081* (0.044)
soil_suitability,NaN,-0.130*** (0.021),-0.132*** (0.021),-0.116*** (0.021)
mean_temp,NaN,0.309*** (0.039),0.315*** (0.040),0.331*** (0.043)
mean_preci,NaN,0.007*** (0.003),0.008*** (0.003),0.009*** (0.003)
dist_100km,NaN,0.028 (0.108),0.020 (0.108),-0.097 (0.114)


In [17]:
# First Stage (same for every regression)
second_unskilled[[1,6,8,24]]

,1,6,8,24
Intercept,0.034 (0.221),-0.757 (0.578),-0.458 (0.587),-3.944*** (1.246)
predicted,-0.397*** (0.089),-0.220*** (0.058),-0.157** (0.061),-0.203*** (0.072)
Adj. R-squared,0.094,0.180,0.208,0.329
std err,0.089004,0.057545,0.061201,0.071758
longitud,0.015*** (0.003),-0.005 (0.005),-0.019*** (0.006),-0.005 (0.014)
latitud,NaN,0.029*** (0.009),0.037*** (0.010),0.098*** (0.022)
soil_suitability,NaN,-0.030** (0.014),-0.024* (0.014),-0.028** (0.014)
mean_temp,NaN,-0.076*** (0.029),-0.117*** (0.031),-0.117*** (0.035)
mean_preci,NaN,-0.004** (0.002),-0.005*** (0.002),-0.005** (0.002)
dist_100km,NaN,0.122** (0.060),0.082 (0.059),0.000 (0.060)


In [18]:
second_low[[1,2,3,24]]

,1,2,3,24
Intercept,-3.180*** (0.239),2.218*** (0.519),2.355*** (0.515),4.635*** (1.425)
predicted,0.751*** (0.096),0.252*** (0.063),0.241*** (0.062),0.256*** (0.082)
Adj. R-squared,0.184,0.197,0.206,0.327
std err,0.096363,0.063272,0.061782,0.08202
longitud,-0.020*** (0.003),-0.012*** (0.003),-0.009*** (0.003),-0.026 (0.016)
latitud,NaN,-0.083*** (0.010),-0.088*** (0.010),-0.122*** (0.025)
soil_suitability,NaN,NaN,0.063*** (0.015),0.056*** (0.016)
mean_temp,NaN,NaN,NaN,0.007 (0.040)
mean_preci,NaN,NaN,NaN,0.000 (0.002)
dist_100km,NaN,NaN,NaN,-0.047 (0.069)


In [19]:
second_medium[[1,2,3,24]]

,1,2,3,24
Intercept,-1.482*** (0.177),-2.090*** (0.380),-2.083*** (0.378),-1.669* (0.990)
predicted,0.156** (0.071),0.212*** (0.046),0.212*** (0.045),0.152*** (0.057)
Adj. R-squared,0.092,0.121,0.128,0.337
std err,0.07118,0.046348,0.045347,0.057018
longitud,-0.015*** (0.002),-0.016*** (0.002),-0.015*** (0.002),0.008 (0.011)
latitud,NaN,0.009 (0.008),0.009 (0.008),-0.024 (0.018)
soil_suitability,NaN,NaN,0.003 (0.011),-0.004 (0.011)
mean_temp,NaN,NaN,NaN,0.099*** (0.028)
mean_preci,NaN,NaN,NaN,0.004*** (0.002)
dist_100km,NaN,NaN,NaN,0.035 (0.048)


In [20]:
second_high[[1,2,3,24]]

,1,2,3,24
Intercept,-5.559*** (0.396),-5.689*** (0.860),-5.584*** (0.858),-0.399 (2.269)
predicted,0.402** (0.160),0.414*** (0.105),0.406*** (0.103),0.529*** (0.131)
Adj. R-squared,0.052,0.066,0.064,0.277
std err,0.159648,0.104872,0.103057,0.130662
longitud,-0.022*** (0.005),-0.022*** (0.005),-0.020*** (0.005),-0.020 (0.026)
latitud,NaN,0.002 (0.017),-0.002 (0.017),-0.105** (0.040)
soil_suitability,NaN,NaN,0.049** (0.024),0.070*** (0.026)
mean_temp,NaN,NaN,NaN,0.072 (0.063)
mean_preci,NaN,NaN,NaN,0.001 (0.004)
dist_100km,NaN,NaN,NaN,-0.073 (0.110)
